## Plug-and-Play Checkpoints: Reproduce Table 1 (NovoMolGen-32M)

> *Run this single cell in the notebook; it downloads the checkpoint, samples
> 30000 valid/canonical SMILES, evaluates the six metrics used in Table 1, and
> renders the result as a tidy dataframe.*

In [ ]:
import os
from pathlib import Path

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HOME"] = "/network/scratch/k/kamran.chitsaz/hf_home"
os.chdir(Path(os.getcwd()).parent)

from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator
import rootutils

rootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)

from src.eval import MoleculeEvaluator
from src.models import generate_valid_smiles, prepare_hf_model
from src.data_loader.utils import load_valid_and_test_data


In [2]:
# 1.  Load pretrained 32 M checkpoint + tokenizer
tokenizer = AutoTokenizer.from_pretrained("chandar-lab/NovoMolGen_32M_SMILES_AtomWise", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("chandar-lab/NovoMolGen_32M_SMILES_AtomWise", trust_remote_code=True)

acc = Accelerator(mixed_precision='bf16')
model = acc.prepare(model)

# 2.  Load pre-computed dataset stats
df_test, df_valid = load_valid_and_test_data("MolGen/ZINC_1B-raw", "175k")
task = MoleculeEvaluator(
    task_names=['unique@1k', 'IntDiv', 'FCD', 'Frag', 'Scaf', 'SNN'],  
    valid_stats=df_valid, 
    test_stats=df_test,
    n_jobs=2
    )

# 3.  Sample 30000 molecules
smiles_list = []
for _ in range(10):
    outputs = model.sample(
        tokenizer=tokenizer, 
        batch_size=3000, 
        max_length=64, 
        temperature=1.0, 
        top_k=1, 
        top_p=0, 
        )
    smiles_list.extend(outputs['SMILES'])

# 4.  Evaluate & display
metrics = task(smiles_list, filter=True, return_valid_index=True)

import pandas as pd
from IPython.display import display

row = {
    "Validity": metrics["validity"],
    "Unique@1k": metrics["unique@1k"],
    "IntDiv":    metrics["IntDiv"],
    "FCD":       metrics["FCD"]["FCD"],
    "SNN":       metrics["SNN"]["SNN"],
    "Frag":      metrics["Frag"]["Frag"],
    "Scaf":      metrics["Scaf"]["Scaf"],
}

df = pd.DataFrame([row]).round(4)
display(df.style.hide(axis="index").set_caption("NovoMolGen-32 M ─ Table 1 metrics"))

2025-08-07 15:16:13.542 | INFO     | src.eval.molecule_evaluation:__call__:169 - Fraction of valid molecules: 1.00
2025-08-07 15:16:23.181 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: unique@1k
2025-08-07 15:16:23.507 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: IntDiv
2025-08-07 15:16:42.291 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: FCD
2025-08-07 15:16:55.677 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: Frag
2025-08-07 15:17:13.951 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: Scaf
2025-08-07 15:17:22.162 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: SNN


Validity,Unique@1k,IntDiv,FCD,SNN,Frag,Scaf
0.999300,1.000000,0.851700,0.040600,0.465500,0.999900,0.518900


### HF Checkpoint

In [4]:
# 1.  Load HF checkpoint (no flash attention)
model = AutoModelForCausalLM.from_pretrained("chandar-lab/NovoMolGen_32M_SMILES_AtomWise", revision='hf-checkpoint', device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("chandar-lab/NovoMolGen_32M_SMILES_AtomWise", revision='hf-checkpoint')
model = prepare_hf_model(model)

# 2.  Load pre-computed dataset stats
df_test, df_valid = load_valid_and_test_data("MolGen/ZINC_1B-raw", "175k")
task = MoleculeEvaluator(
    task_names=['unique@1k', 'IntDiv', 'FCD', 'Frag', 'Scaf', 'SNN'],  
    valid_stats=df_valid, 
    test_stats=df_test,
    n_jobs=2
    )

# 3.  Sample 30000 molecules
smiles_list = []
for _ in range(10):
    outputs = model.sample(
        tokenizer=tokenizer, 
        batch_size=1000, 
        max_length=64, 
        temperature=1.0, 
        top_k=0, 
        top_p=1.0,
        do_sample=True, 
        )
    smiles_list.extend(outputs['SMILES'])

# 4.  Evaluate & display
metrics = task(smiles_list, filter=True, return_valid_index=True)

import pandas as pd
from IPython.display import display

row = {
    "Validity": metrics["validity"],
    "Unique@1k": metrics["unique@1k"],
    "IntDiv":    metrics["IntDiv"],
    "FCD":       metrics["FCD"]["FCD"],
    "SNN":       metrics["SNN"]["SNN"],
    "Frag":      metrics["Frag"]["Frag"],
    "Scaf":      metrics["Scaf"]["Scaf"],
}

df = pd.DataFrame([row]).round(4)
display(df.style.hide(axis="index").set_caption("NovoMolGen-32 M ─ Table 1 metrics"))

2025-09-02 13:48:23.553 | INFO     | src.eval.molecule_evaluation:__call__:169 - Fraction of valid molecules: 1.00
2025-09-02 13:48:27.322 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: unique@1k
2025-09-02 13:48:27.590 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: IntDiv
2025-09-02 13:48:32.663 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: FCD
2025-09-02 13:48:38.891 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: Frag
2025-09-02 13:48:45.196 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: Scaf
2025-09-02 13:48:48.459 | INFO     | src.eval.molecule_evaluation:__call__:185 - Evaluating task: SNN


Validity,Unique@1k,IntDiv,FCD,SNN,Frag,Scaf
0.999500,1.000000,0.851100,0.097900,0.465800,0.999800,0.315000
